*This notebook gets the samknows metadata snapshot from `metadata_snapshot` table; finds the asn for each public_ip of the probe; inserts (probeid, asn) dataframe in `metadata_snapshot_extension` table.*
- - - 

In [13]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [28]:
DB_LOCATION = 'lastmile.db'
METADATA_SNAPSHOT = 'metadata_snapshot'
METADATA_SNAPSHOT_EXTENSIONS = 'metadata_snapshot_extensions'

In [30]:
con = sqlite3.connect(DB_LOCATION)

In [16]:
query = '''SELECT    id as probeid, public_ip
           FROM      %s
        '''%(METADATA_SNAPSHOT)

df = pd.read_sql(query, con)

In [17]:
df.count()

probeid      2230
public_ip    2230
dtype: int64

In [18]:
def get_json_resource_from_absolute_uri(url, query_params):
    try: res = requests.get(url, params = query_params)
    except Exception as e: print(e, file=sys.stderr)
    else:
        try: res_json = res.json()
        except Exception as e: print(e, file=sys.stderr)
        else: 
            return res_json

In [19]:
def get_asn_from_endpoint(endpoint):
    asn = holder = None
    base_uri = 'https://stat.ripe.net'; url = '%s/data/prefix-overview/data.json'%base_uri
    params = {'resource' : endpoint}
    try: res = get_json_resource_from_absolute_uri(url, params)
    except Exception as e: print(e, file=sys.stderr); return None
    try:
        asns = []
        for item in res['data']['asns']:
            asn = item['asn']; holder = item['holder']
            asns.append(asn)
    except Exception as e: print(e, file=sys.stderr)
    else: 
        asns = set(asns)
        print('.', end=''); import sys; sys.stdout.flush()
        if len(asns) == 1: return asns.pop()
        else: return None
    return 

In [20]:
df['asn'] = df['public_ip'].apply(get_asn_from_endpoint)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [21]:
df.head()

,probeid,public_ip,asn
0,1457,62.255.129.51,5089
1,2175,82.10.83.53,5089
2,5079,86.128.67.7,2856
3,5136,86.171.221.48,2856
4,14190,178.78.87.102,12390


In [22]:
df.count()

probeid      2230
public_ip    2230
asn          2230
dtype: int64

In [23]:
cur = con.execute('pragma foreign_keys=ON')

In [24]:
index_label = ['probeid']
df = df.set_index(index_label)

In [25]:
del df['public_ip']
df.head()

,asn
probeid,
1457,5089
2175,5089
5079,2856
5136,2856
14190,12390


In [31]:
df.to_sql(  '%s'%METADATA_SNAPSHOT_EXTENSIONS
          , con
          , flavor='sqlite'
          , if_exists = 'append'
          , index_label = index_label
         )

In [32]:
con.commit()
con.close()